In [34]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import calendar as cal
import numpy as np
import pandas as pd
import pandas_profiling as pp
import calendar as cal
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

from tsfresh import extract_features,extract_relevant_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters

from tpot import TPOTRegressor
from boruta import BorutaPy
import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

from IPython.core.display import HTML
%matplotlib inline

In [2]:
df = pd.read_csv('https://s3.us-east-2.amazonaws.com/ads-demo1/E_Dataset.csv',parse_dates=['date'])

In [3]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
df['year'] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter
df['month'] = df['date'].dt.month
df['weekNumberInYear'] = df['date'].dt.week
df['dayOfMonth'] = df['date'].dt.day
df['dayInWeek'] = df['date'].dt.dayofweek
df['hourOfDay'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

df['WeekendFlag'] = df['dayInWeek'].apply(lambda x : 1 if (x == 'Saturday') or (x == 'Sunday') else 0).astype('int64')
df['TotalSecondsMidnight'] = (df['date'].dt.hour * 3600) + (df['date'].dt.minute * 60) + (df['date'].dt.second)

dayInWeek = pd.get_dummies(df.dayInWeek,prefix='dayInWeek').astype('int64')
weekendflag = pd.get_dummies(df.WeekendFlag,prefix='weekendflag').astype('int64')

df = pd.concat([df,dayInWeek,weekendflag],axis=1)

In [5]:
rmse_dict = {}

def rmse(correct,estimated):
    rmse_val = np.sqrt(mean_squared_error(correct,estimated)) 
    return rmse_val

def calc_error_metric(name, model, X_train, y_train, X_test, y_test):
    
    y_train_predicted = model.predict(X_train)
    y_test_predicted = model.predict(X_test)
        
    #MAE, RMS, MAPE, R2
    
    r2_train = r2_score(y_train, y_train_predicted)
    r2_test = r2_score(y_test, y_test_predicted)
    
    rms_train = rmse(y_train, y_train_predicted)
    rms_test = rmse(y_test, y_test_predicted)
        
    mae_train = mean_absolute_error(y_train, y_train_predicted)
    mae_test = mean_absolute_error(y_test, y_test_predicted)
        
    mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
    mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
    
    rmse_dict[name] = rms_test
        
    df_local = pd.DataFrame({'Model':[name],
                            'r2_train': [r2_train],
                            'r2_test': [r2_test],
                            'rms_train':[rms_train], 
                            'rms_test': [rms_test],
                            'mae_train': [mae_train],
                            'mae_test': [mae_test],
                            'mape_train':[mape_train],
                            'mape_test':[mape_test]})

    error_metric = pd.concat([df_local])
 
    return error_metric

In [6]:
display(HTML(df.head().to_html()))

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,year,quarter,month,weekNumberInYear,dayOfMonth,dayInWeek,hourOfDay,minute,WeekendFlag,TotalSecondsMidnight,dayInWeek_0,dayInWeek_1,dayInWeek_2,dayInWeek_3,dayInWeek_4,dayInWeek_5,dayInWeek_6,weekendflag_0
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433,2016,1,1,2,11,0,17,0,0,61200,1,0,0,0,0,0,0,1
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195,2016,1,1,2,11,0,17,10,0,61800,1,0,0,0,0,0,0,1
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668,2016,1,1,2,11,0,17,20,0,62400,1,0,0,0,0,0,0,1
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389,2016,1,1,2,11,0,17,30,0,63000,1,0,0,0,0,0,0,1
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097,2016,1,1,2,11,0,17,40,0,63600,1,0,0,0,0,0,0,1


In [7]:
df_1 = df.drop(['date','rv1','rv2','dayInWeek'],axis=1)

In [8]:
X = df_1.drop(['Appliances'],axis=1)
y = df_1['Appliances']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

## 1. BORUTA

In [10]:
X_boruta = df_1
X_boruta = X_boruta.drop(['Appliances'],axis=1)
X_boruta = X_boruta.values

In [11]:
y_boruta = df_1
y_boruta = y_boruta['Appliances']
y_boruta = y.values

In [12]:
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
y_boruta = y_boruta.ravel()

# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf1 = RandomForestRegressor(n_jobs=-1, max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf1, n_estimators='auto', verbose=5, random_state=1,max_iter=50)

# find all relevant features - 5 features should be selected
feat_selector.fit(X_boruta,y_boruta)

# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	2 / 50
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	3 / 50
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	4 / 50
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	5 / 50
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	6 / 50
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	7 / 50
Confirmed: 	0
Tentative: 	42
Rejected: 	0
Iteration: 	8 / 50
Confirmed: 	8
Tentative: 	16
Rejected: 	18
Iteration: 	9 / 50
Confirmed: 	8
Tentative: 	16
Rejected: 	18
Iteration: 	10 / 50
Confirmed: 	8
Tentative: 	16
Rejected: 	18
Iteration: 	11 / 50
Confirmed: 	8
Tentative: 	16
Rejected: 	18
Iteration: 	12 / 50
Confirmed: 	8
Tentative: 	16
Rejected: 	18
Iteration: 	13 / 50
Confirmed: 	8
Tentative: 	15
Rejected: 	19
Iteration: 	14 / 50
Confirmed: 	8
Tentative: 	15
Rejected: 	19
Iteration: 	15 / 50
Confirmed: 	8
Tentative: 	15
Rejected: 	19
Iteration: 	16 / 50
Confirmed: 	9
Tentative: 	12
Rejected: 	21
Iteratio

array([ 1, 12, 14,  6,  1,  1,  1,  7,  1,  1,  1,  9,  2,  5,  1,  1,  8,
        1, 10,  5,  1,  3,  2, 12,  3, 25, 21,  1,  1, 15,  1, 16, 25,  1,
       20, 19, 18, 23, 14, 22, 18, 25])

In [13]:
boruta_feature_ranking = zip(list(X.columns),list(feat_selector.ranking_))
boruta_feature_ranking = set(boruta_feature_ranking)
print(boruta_feature_ranking)

{('T4', 7), ('minute', 16), ('dayInWeek_1', 19), ('lights', 1), ('RH_out', 3), ('T3', 1), ('RH_7', 1), ('dayOfMonth', 15), ('T9', 1), ('WeekendFlag', 25), ('dayInWeek_5', 22), ('T_out', 5), ('RH_6', 2), ('Press_mm_hg', 1), ('Windspeed', 2), ('TotalSecondsMidnight', 1), ('T1', 12), ('dayInWeek_2', 18), ('T2', 6), ('T8', 1), ('dayInWeek_0', 20), ('RH_9', 10), ('dayInWeek_4', 14), ('T6', 9), ('Visibility', 12), ('RH_8', 8), ('year', 25), ('weekendflag_0', 25), ('dayInWeek_3', 23), ('RH_4', 1), ('month', 1), ('weekNumberInYear', 1), ('RH_1', 14), ('RH_3', 1), ('T5', 1), ('T7', 5), ('Tdewpoint', 3), ('hourOfDay', 1), ('dayInWeek_6', 18), ('RH_2', 1), ('quarter', 21), ('RH_5', 1)}


In [14]:
boruta_features = ['RH_5','T8','RH_7','hourOfDay','month','T3','Press_mm_hg',
                   'TotalSecondsMidnight','weekNumberInYear','T9','lights','RH_2','T5','RH_4','RH_3','Appliances']

boruta_df = df_1[boruta_features]

X_from_boruta = boruta_df.drop('Appliances',axis=1)
y_from_boruta = boruta_df['Appliances']

In [15]:
X_train_boruta, X_test_boruta, y_train_boruta, y_test_boruta = train_test_split(X_from_boruta, y_from_boruta, 
                                                                                test_size=0.30, random_state=42)

In [16]:
rf_boruta = RandomForestRegressor(bootstrap=True, max_depth=30, n_estimators=30,n_jobs=-1)
rf_boruta.fit(X_train_boruta,y_train_boruta)
calc_error_metric('Random Forrest_Boruta',rf_boruta, X_train_boruta, y_train_boruta, X_test_boruta, y_test_boruta)

,Model,r2_train,r2_test,rms_train,rms_test,mae_train,mae_test,mape_train,mape_test
0,Random Forrest_Boruta,0.927821,0.531899,27.660386,69.445427,12.773232,32.990377,12.546246,32.933694


## 2. TSFRESH

In [17]:
display(HTML(df_1.head().to_html()))

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,year,quarter,month,weekNumberInYear,dayOfMonth,hourOfDay,minute,WeekendFlag,TotalSecondsMidnight,dayInWeek_0,dayInWeek_1,dayInWeek_2,dayInWeek_3,dayInWeek_4,dayInWeek_5,dayInWeek_6,weekendflag_0
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,2016,1,1,2,11,17,0,0,61200,1,0,0,0,0,0,0,1
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,2016,1,1,2,11,17,10,0,61800,1,0,0,0,0,0,0,1
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,2016,1,1,2,11,17,20,0,62400,1,0,0,0,0,0,0,1
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,2016,1,1,2,11,17,30,0,63000,1,0,0,0,0,0,0,1
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,2016,1,1,2,11,17,40,0,63600,1,0,0,0,0,0,0,1


In [18]:
ts_df = df_1

In [19]:
ts_df = ts_df.reset_index()
ts_df['date'] = df['date']

In [20]:
X_tsfresh = ts_df.drop('Appliances',axis=1)
y_tsfresh = ts_df['Appliances']
y_tsfresh.index = ts_df['date']

In [21]:
extracted_features = extract_features(ts_df, column_id="date",
                                      show_warnings=False, default_fc_parameters=MinimalFCParameters())

Feature Extraction: 100%|██████████| 10/10 [02:24<00:00, 11.96s/it]


In [22]:
impute(extracted_features)
features_filtered = select_features(extracted_features,y_tsfresh)

In [23]:
X_from_tsfresh = features_filtered

X_train_ts, X_test_ts, y_train_ts, y_test_ts = train_test_split(X_from_tsfresh, y_tsfresh, 
                                                                                test_size=0.30)

In [24]:
rf_ts = RandomForestRegressor(bootstrap=True, max_depth=30, n_estimators=30,n_jobs=-1)
rf_ts.fit(X_train_ts,y_train_ts)
calc_error_metric('Random Forrest_TSFresh',rf_ts, X_train_ts, y_train_ts, X_test_ts, y_test_ts)

,Model,r2_train,r2_test,rms_train,rms_test,mae_train,mae_test,mape_train,mape_test
0,Random Forrest_TSFresh,0.999944,0.999325,0.769781,2.669877,0.018508,0.069414,0.002205,0.007665


## 3. TPOT

In [25]:
tpot = TPOTRegressor(generations=2, population_size=50,
                     offspring_size=None,
                     mutation_rate=0.9,
                     verbosity=3,cv=2,n_jobs=-1)

tpot.fit(X_train, y_train)
tpot.export('tpot_energy_pipeline.py')

28 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 X contains negative values.
_pre_test decorator: _random_mutation_operator: num_test=0 __init__() got an unexpected keyword argument 'max_depth'
_pre_test decorator: _random_mutation_operator: num_test=0 __init__() got an unexpected keyword argument 'max_depth'
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False
Generation 1 - Current Pareto front scores:
-1	-5847.517134304666	RandomForestRegressor(input_matrix, RandomForestRegressor__bootstrap=False, RandomForestRegressor__max_features=0.3, RandomForestRegressor__min_samples_leaf=1, RandomForestRegressor__min_samples_split=11, RandomForestRegressor__n_estimators=100)

_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a m

True

In [26]:
print('The RMSE of TPOT Regressor is {}'.format(tpot.score(X_test,y_test)))

The RMSE of TPOT Regressor is -4338.882742575104


## 4. Feature Tools

In [27]:
df_feature = df
df_feature = df_feature.set_index(df['date'])

In [28]:
df_feature['dayInWeek'] = df_feature['dayInWeek'].apply(str)
df_feature['hourOfDay'] = df_feature['hourOfDay'].apply(str)
df_feature['minute'] = df_feature['minute'].apply(str)
df_feature['period']=df_feature[['dayInWeek','hourOfDay','minute']].apply(lambda x:''.join(x),axis=1)

In [29]:
df_feature = df_feature.drop(['rv1','rv2'],axis=1)
display(HTML(df_feature.head().to_html()))

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,year,quarter,month,weekNumberInYear,dayOfMonth,dayInWeek,hourOfDay,minute,WeekendFlag,TotalSecondsMidnight,dayInWeek_0,dayInWeek_1,dayInWeek_2,dayInWeek_3,dayInWeek_4,dayInWeek_5,dayInWeek_6,weekendflag_0,period
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,2016,1,1,2,11,0,17,0,0,61200,1,0,0,0,0,0,0,1,0170
2016-01-11 17:10:00,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,2016,1,1,2,11,0,17,10,0,61800,1,0,0,0,0,0,0,1,01710
2016-01-11 17:20:00,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,2016,1,1,2,11,0,17,20,0,62400,1,0,0,0,0,0,0,1,01720
2016-01-11 17:30:00,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,2016,1,1,2,11,0,17,30,0,63000,1,0,0,0,0,0,0,1,01730
2016-01-11 17:40:00,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,2016,1,1,2,11,0,17,40,0,63600,1,0,0,0,0,0,0,1,01740


In [30]:
phase_featuretools = df_feature.groupby(['date','month','dayInWeek','hourOfDay','dayOfMonth','minute','period']).mean()
phase_featuretools = phase_featuretools.reset_index()
phase_featuretools['dayInWeek']=phase_featuretools['dayInWeek'].apply(int)
phase_featuretools['hourOfDay']=phase_featuretools['hourOfDay'].apply(int)
phase_featuretools['minute']=phase_featuretools['minute'].apply(int)
phase_featuretools['period']=phase_featuretools['period'].apply(int)
phase_featuretools['date'] = pd.to_datetime(phase_featuretools['date'])
phase_featuretools['Press_mm_hg'] = np.log(phase_featuretools['Press_mm_hg'])
phase_featuretools['Visibility'] = np.log(phase_featuretools['Visibility'])

In [31]:
y_featuretools = phase_featuretools[['date','Appliances']]
X_featuretools = phase_featuretools.drop(['Appliances'],axis=1)
entities ={"appliances" :(y_featuretools,"date"),
          "rest" :(X_featuretools,"date")}
relationships = [("appliances","date","rest","date")]

In [32]:
feature_matrix_app,features_defs = ft.dfs(entities=entities,relationships=relationships,target_entity="appliances")

In [33]:
features_defs

[<Feature: Appliances>,
 <Feature: SUM(rest.month)>,
 <Feature: SUM(rest.dayInWeek)>,
 <Feature: SUM(rest.hourOfDay)>,
 <Feature: SUM(rest.dayOfMonth)>,
 <Feature: SUM(rest.minute)>,
 <Feature: SUM(rest.period)>,
 <Feature: SUM(rest.lights)>,
 <Feature: SUM(rest.T1)>,
 <Feature: SUM(rest.RH_1)>,
 <Feature: SUM(rest.T2)>,
 <Feature: SUM(rest.RH_2)>,
 <Feature: SUM(rest.T3)>,
 <Feature: SUM(rest.RH_3)>,
 <Feature: SUM(rest.T4)>,
 <Feature: SUM(rest.RH_4)>,
 <Feature: SUM(rest.T5)>,
 <Feature: SUM(rest.RH_5)>,
 <Feature: SUM(rest.T6)>,
 <Feature: SUM(rest.RH_6)>,
 <Feature: SUM(rest.T7)>,
 <Feature: SUM(rest.RH_7)>,
 <Feature: SUM(rest.T8)>,
 <Feature: SUM(rest.RH_8)>,
 <Feature: SUM(rest.T9)>,
 <Feature: SUM(rest.RH_9)>,
 <Feature: SUM(rest.T_out)>,
 <Feature: SUM(rest.Press_mm_hg)>,
 <Feature: SUM(rest.RH_out)>,
 <Feature: SUM(rest.Windspeed)>,
 <Feature: SUM(rest.Visibility)>,
 <Feature: SUM(rest.Tdewpoint)>,
 <Feature: SUM(rest.year)>,
 <Feature: SUM(rest.quarter)>,
 <Feature: SUM(res